# 近似求导

In [2]:
def f(x):
    return 3. * x ** 2 + 2. * x - 1
#近视求导，x移动eps单位，也就是离自己很近的一个点的切线 符号求导是6x+2
def approximate_derivative(f, x, eps=1e-6):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(approximate_derivative(f, 1.))

7.999999999785956


In [3]:
def g(x1, x2):
    return (x1 + 5) * (x2 ** 2) 

In [4]:
# 对多变量函数求偏导数
def partial_derivative(f, x1, x2, var, eps=1e-6):
    """
    计算多变量函数的偏导数
    
    参数:
        f: 多变量函数
        x1, x2: 计算偏导数的点坐标
        var: 对哪个变量求偏导，'x1'或'x2'
        eps: 微小变化量
    
    返回:
        指定变量的偏导数值
    """
    if var == 'x1':
        return (f(x1 + eps, x2) - f(x1 - eps, x2)) / (2 * eps)
    elif var == 'x2':
        return (f(x1, x2 + eps) - f(x1, x2 - eps)) / (2 * eps)
    else:
        raise ValueError("var参数必须是'x1'或'x2'")

# 计算g函数在点(1, 2)处对x1和x2的偏导数
x1_point, x2_point = 1, 2

# 对x1求偏导数：∂g/∂x1 = x2^2
derivative_x1 = partial_derivative(g, x1_point, x2_point, 'x1')
print(f"在点({x1_point}, {x2_point})处，g对x1的偏导数为: {derivative_x1}")
# 理论值为 x2^2 = 2^2 = 4

# 对x2求偏导数：∂g/∂x2 = 2(x1+5)x2
derivative_x2 = partial_derivative(g, x1_point, x2_point, 'x2')
print(f"在点({x1_point}, {x2_point})处，g对x2的偏导数为: {derivative_x2}")
# 理论值为 2(x1+5)x2 = 2(1+5)*2 = 24

在点(1, 2)处，g对x1的偏导数为: 4.000000000559112
在点(1, 2)处，g对x2的偏导数为: 24.000000001578314


## torch 近似求导

In [5]:
import torch

x1 = torch.tensor([1.], requires_grad=True) # 创建一个标量tensor，并设置requires_grad=True以便进行自动求导
x2 = torch.tensor([2.], requires_grad=True)
y = g(x1, x2) #前向计算是 构建计算图的过程
    
(dy_dx1,) = torch.autograd.grad(y, x1,retain_graph=True) # 求导数，retain_graph=True 表示保留计算图，以便进行多次求导
print(dy_dx1)



tensor([4.])


In [6]:
try:
    (dy_dx1,) = torch.autograd.grad(y, x1,retain_graph=True) # autograd.grad()函数是用于计算张量的梯度
except Exception as e:
    print(e)

print(dy_dx1)

tensor([4.])


In [7]:
x1 = torch.tensor([2.], requires_grad=True) 
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2)

# 求偏导数
dy_dx1, dy_dx2 = torch.autograd.grad(y, [x1, x2],retain_graph=True)


print(dy_dx1, dy_dx2)

tensor([9.]) tensor([42.])


# 使用backward

In [8]:
x1 = torch.tensor([2.], requires_grad=True)
x2 = torch.tensor([3.], requires_grad=True)
y = g(x1, x2) #前向计算是 构建计算图的过程

# 使用backward求导
y.backward()

# 获取梯度
print(x1.grad, x2.grad)

tensor([9.]) tensor([42.])


# 理解优化器的梯度清空 optimizer.zero_grad()，和    optimizer.step()

In [9]:
#模拟梯度下降算法 SGD
import torch
learning_rate = 0.3
x = torch.tensor(2.0, requires_grad=True)
for _ in range(100):
    z = f(x) #前向传播
    if _>0:
        x.grad.zero_() # x.grad -= x.grad, x.grad = 0,梯度清零 下划线代表在原始数据上操作
    z.backward()
    x.data -= learning_rate * x.grad
    # x.data.sub_(learning_rate * x.grad) # x -= learning_rate * x.grad，这里就等价于optimizer.step()
   
print(x)
print(f(x))

tensor(-0.3333, requires_grad=True)
tensor(-1.3333, grad_fn=<SubBackward0>)


In [10]:
# 使用PyTorch的优化器实现梯度下降
import torch
import torch.optim as optim

# 定义初始值和学习率
x = torch.tensor(2.0, requires_grad=True)
learning_rate = 0.3

# 创建SGD优化器
optimizer = optim.SGD([x], lr=learning_rate)

# 进行100次迭代优化
for _ in range(100):
    # 计算函数值（前向传播）
    z = f(x)
    
    # 清空梯度
    optimizer.zero_grad()
    
    # 反向传播计算梯度
    z.backward()
    
    # 更新参数
    optimizer.step()

print(f"优化后的x值: {x}")
print(f"对应的函数值: {f(x)}")

优化后的x值: -0.3333333432674408
对应的函数值: -1.3333332538604736
